In [29]:
import psycopg2
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

zona = 'tornavacas'

params = config()
conn = psycopg2.connect(**params)

sup_df = pd.DataFrame({'parcela': [],'sup (ha)':[]})
stats_df = pd.DataFrame({'parcela': [], 
                         'altura máxima (m)': [], 
                         'altura mínima (m)': [], 
                         'altura media (m)': [], 
                           'pendiente (%)': []})

orientaciones_df = pd.DataFrame({'parcela': [], 'orientacion': []})

limites_df = pd.DataFrame({'parcela': [],
                          'Norte': [],
                          'Sur': [],
                          'Este': [],
                          'Oeste': []})

habitats_parcelas = []

files = !ls ../FINALES/PARCELAS/TORNAVACAS/*shp
for index, parcela in enumerate(files):
    
    # SUPERFICIE
    
    nombre = parcela.split('/')[-1].split('.')[0]
    sup = pd.read_sql('''
                SELECT
                ST_Area(ST_Transform(geom, 25830)) / 10000 as sup
                FROM {}
                '''.format(nombre), conn)
    df = pd.DataFrame({'sup (ha)': [sup['sup'][0]], 'parcela': [str(index + 1)]})
    sup_df = sup_df.append(df, ignore_index=True)

    #PENDIENTES Y ALTITUD
    
    pendientes = pd.read_sql('''
                SELECT
                (ST_SummaryStats(St_Union(ST_Clip(rast, st_transform(geom, 25830))))).*
                from {}, pendientes25
                '''.format(nombre), conn)

    stats = pd.read_sql('''
                SELECT
                (ST_SummaryStats(St_Union(ST_Clip(rast, st_transform(geom, 25830))))).*
                from {}, mdt25
                '''.format(nombre), conn)
    
    df = pd.DataFrame({'altura máxima (m)': [stats['max'][0]], 
                       'altura mínima (m)': [stats['min'][0]], 
                       'altura media (m)': [stats['mean'][0]],
                        'parcela': str(index + 1),
                          'pendiente (%)': [pendientes['mean'][0]]})
    stats_df = stats_df.append(df, ignore_index=True)

    # ORIENTACIONES
    
    orientaciones = pd.read_sql('''
                SELECT
                (ST_SummaryStats(St_Union(ST_Clip(rast, st_transform(geom, 25830))))).*
                from {}, orientaciones25
                '''.format(nombre), conn)    
    df = pd.DataFrame({'parcela': str(index + 1), 'orientacion': [orientaciones['mean'][0]]})
    orientaciones_df = orientaciones_df.append(df, ignore_index=True)
    
    df = pd.DataFrame({'parcela': [str(index + 1)]})
    limites_df = limites_df.append(df, ignore_index=True)
    
    # HABITATS
    
    habitats = pd.read_sql('''
                SELECT
                    codue as codigo,
                    descripcio, 
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / 10000 as superficie,
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / st_area(st_transform(a.geom, 23030)) * 100 as porcentaje
                FROM atlashabitats2005_extremadura b, {} a 
                WHERE st_intersects(st_transform(a.geom, 23030), b.geom)
                '''.format(nombre), conn)
    habitats_parcelas.append(habitats)
    
prescripcion = pd.DataFrame({
    'Factores meteorológicos': ['Temperatura', 'Humedad Relativa', 'Viento', 'Humedad del suelo', 'HCFM', 'Nº días sin llover'],
    'Rango base': ['5-20 ºC (heladas previas)', '30-60%', '5-15 km/h', '50-100%', '5-11%', '5-25 días'],
    'Rango práctico': ['6-21 ºC', '27 - 64 %%', '0-25 km/h', 'Húmedo', '6-10%', '2-12 días']
})
    
conn.close()

env = Environment(loader=FileSystemLoader('static/plan_quemas'))
template =env.get_template('base.html')
output = template.render(
    zona = zona,
    sup = sup_df,
    stats = stats_df,
    orientaciones = orientaciones_df,
    limites = limites_df,
    prescripcion = prescripcion,
    habitats = habitats_parcelas
)

with open('static/plan_quemas/plan_quemas.html', 'w') as f:
    f.write(output)